In [1]:
from twitch_listener import utils
from twitch_listener import listener

Some weights of the model checkpoint at veb/twitch-bert-base-cased-pytorch were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at veb/twitch-bert-base-cased-pytorch and are newly initialized: ['bert.pooler.dense.bias', '

In [2]:
# Connect to Twitch
# bot = listener.connect_twitch('twitch_pioneers', 
#                              'oauth:lolbsmiac5expvax1iqiysfo18hqi2', 
#                              '6resxnu2ehi2ggn0kgpue6g12bxtlw',
#                              'zkekfhyhzhdl8ltxs6b8jdez7b2i6b')

nickname = 'twitch_pioneers'
oauth_chat = 'oauth:lolbsmiac5expvax1iqiysfo18hqi2'
client_id = '6resxnu2ehi2ggn0kgpue6g12bxtlw'
oauth_api = 'Bearer y5o8h4s2fljc1s632ylfrb540npcs0'


bot = listener.connect_twitch(nickname, 
                             oauth_chat, 
                             client_id,
                             oauth_api)

chat_file_path = '/home/w210/Twitch-chat-pioneers/chat_listener'

# List of channels to connect to
# Channel names must be lowercase
channels_to_listen_to = ['cryocells','jingggxd','gmhikaru','lirik']

In [3]:
#returns list of live channels
channels_to_listen_to = utils.is_live(channels_to_listen_to)

channels_to_listen_to = utils.get_broadcast_id(channels_to_listen_to, client_id, oauth_api)
print(channels_to_listen_to)

{'cryocells': '432532052', 'jingggxd': '136397315'}


In [4]:
# Scrape live chat data into raw log files. (Duration is seconds)
bot.listen(channels_to_listen_to, duration = 30, until_offline = True, debug = False) 

# Convert log files into .CSV format
#bot.parse_logs(timestamp = True, file_path = chat_file_path)

# Generate adjacency matrix
#bot.adj_matrix(weighted = False, matrix_name = "streamer_network.csv")

Channels Offline


In [ ]:
#Pass to function
from load_sentiment_model import load_model
from transformers import BertTokenizer
import torch
import sqlite3

PRE_TRAINED_MODEL_NAME = "veb/twitch-bert-base-cased-pytorch" #'models/veb/twitch-bert-base-cased-finetuned'
MAX_LEN = 160
class_names = ['negative', 'neutral', 'positive']
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
model = load_model()
conn = sqlite3.connect('/home/w210/Twitch-chat-pioneers/src/front_end/db.sqlite3')
cursor_obj = conn.cursor()
cursor_obj.execute("""select username, channel_name, date, message_text from chats_table_demo
                    where message_sentiment is NULL """)
output = cursor_obj.fetchall()
for review_text in output:
    #Get sentiment for each message
    sentiment = utils.message_sentiment(review_text[3],
                                  tokenizer,
                                  model, 
                                  class_names, 
                                  PRE_TRAINED_MODEL_NAME,MAX_LEN = 160)
    #Update table with sentiments
    cursor_obj.execute("""UPDATE chats_table_demo
                        SET message_sentiment = {}
                        WHERE username = {} and 
                        channel_name = {} and
                        date = {};""".format(sentiment,
                                                "\'" + review_text[0] + "\'",
                                                 "\'" + review_text[1] + "\'",
                                                 "\'" + review_text[2] + "\'" ))

conn.commit()
conn.close()  
    

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/w210/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2301: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [2]:
import sqlite3


In [6]:
#test = conn.execute('''drop table if exists chats_table_demo''')
#test = conn.execute('''create table chats_table_test as select * from chats_table_sentiment limit 100''')
conn = sqlite3.connect('/home/w210/Twitch-chat-pioneers/src/front_end/db.sqlite3')
test = conn.execute('''select channel_name,stream_topic,  count(*), max(stream_length) from chats_table_demo group by channel_name, stream_topic''')
output = test.fetchall()
conn.close()
output

[('a2guapo', 'VALORANT', 146, 15),
 ('cryocells', 'VALORANT', 665, 39),
 ('faide', 'Apex Legends', 951, 18),
 ('gmhikaru', 'Just Chatting', 683, 169),
 ('jingggxd', 'VALORANT', 1351, 58),
 ('lirik', 'Evolve Stage 2', 7426, 178),
 ('lirik', 'Just Chatting', 31576, 98),
 ('nats', 'VALORANT', 293, 23),
 ('shivfps', 'Apex Legends', 3033, 174),
 ('sweetdreams', 'Apex Legends', 3414, 177),
 ('xeppaa', 'VALORANT', 275, 7)]

In [2]:
conn = sqlite3.connect('/home/w210/Twitch-chat-pioneers/src/front_end/db.sqlite3')
test = conn.execute('''select channel_name,  count(*) from chats_table_demo group by channel_name ''')
output = test.fetchall()
conn.close()
output

[('amouranth',
  'ð\x9f\x8d\x91BIKINI MOIST MONDAYSð\x9f\x92¦ type !s for my social media',
  'Pools, Hot Tubs, and Beaches'),
 ('tarik', 'BACK ON IT | Twitter @tarik', 'VALORANT'),
 ('iitztimmy',
  'Practicing for Rivals // Follow my socials @iiTzTimmy',
  'Apex Legends'),
 ('nickmercs',
  "Warzone Fraggin' | Code: MFAM | @NICKMERCS",
  'Call of Duty: Warzone'),
 ('iitztimmy', 'apexge // Follow my socials @iiTzTimmy', 'Apex Legends'),
 ('nickmercs', 'Hibernating | Code: MFAM | @NICKMERCS', 'Apex Legends')]

In [24]:
test = conn.execute('''select count(*) as messages, 
                        avg(viewer_count) as avg_viewers,
                        avg(subscriber_count) as avg_subscribers
                        from chats_table
                        group by channel_name, stream_topic, stream_date
                        ''')
test.fetchall()

[(28, 9235.285714285714, 73285.5),
 (21, 9521.142857142857, 71950.85714285714),
 (101, 20006.465346534653, 26255.48514851485)]

In [ ]:
INSERT INTO chats_table(
                                          date, 
                                          stream_date,
                                          stream_length,
                                          username, 
                                          message_text, 
                                          channel_name,
                                          stream_topic,
                                          stream_title,
                                          chatter_count,
                                          viewer_count,
                                          follower_count,
                                          subscriber_count,
                                          stream_id,
                                         avg_sentiment